# Introduction

This is a basic example of using TOAST interactively for LiteBIRD simulations.  This uses an extra package to help displaying things in the notebook.  You can install that with `pip install wurlitzer` and restart this notebook kernel.

In [ ]:
# Built-in modules
import sys
import os
from datetime import datetime

# External modules
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
import healpy as hp

# LiteBIRD and TOAST tools

import toast
import toast.ops
from toast import schedule_sim_satellite as schedulesim
from toast import pixels_io as pio

import litebirdtask as lbt
from litebirdtask import vis as lbtv
from litebirdtask import ops as lbtops


# Capture C++ output in the jupyter cells
%load_ext wurlitzer

# Display inline plots
%matplotlib inline

## Instrument Data

Specify the location of the instrument / hardware file you downloaded from the wiki (See https://wiki.kek.jp/pages/viewpage.action?pageId=150667506)

In [ ]:
hwfile = "/home/kisner/git/litebird/litebird_model_2021-02-15T22:12:51.toml.gz"

### Load the Hardware Model

This loads the full instrument model:

In [ ]:
hw = lbt.Hardware(path=hwfile)

### Select Detectors

The file you download from the wiki has **all** detectors.  For this example, we will select just a few of them.

In [ ]:
lfhw = hw.select(
    match={
        "wafer": ["L00",],
        "band": ".*040",
        "pixel": "00."
    }
)

We can see which detectors were selected:

In [ ]:
lbtv.summary_text(lfhw)

## Observing Schedule

Before running the simulation we need to create an "observing schedule".  This is a simple model of stable science scans separated by optional "gaps".  For this example we will make contiguous scans with no gaps.  Here we make a schedule for 1 day of observing, with one-hour stable science scans:

In [ ]:
schedule = schedulesim.create_satellite_schedule(
    prefix="LB_",
    mission_start=datetime.fromisoformat("2030-07-07T07:07:07+00:00"),
    observation_time=60 * u.minute,
    gap_time=0 * u.minute,
    num_observations=24, # 1 day x 24 obs per day
    prec_period=3.2058 * u.hour, # From IMOv1 wiki page
    spin_period=20 * u.minute, # From IMOv1 wiki, 0.05 RPM = 20 minutes
)

In [ ]:
print(schedule)

We can also write / read this schedule to disk.

## Simulate the Scanning

Next we are going to run a LiteBIRD scanning simulation.  We start with an empty TOAST data container:

In [ ]:
data = toast.Data()
print(data)

Now we will create a LiteBIRD scanning "Operator" and apply it to the data.  We can always see the help for an operator before we use it:

In [ ]:
# This will pop up a help window
#?lbtops.SimScan

In [ ]:
# Create the operator

sim_scan = lbtops.SimScan(
    hardware=lfhw,
    schedule=schedule,
    hwp_angle="hwp_angle",
    hwp_rpm=46.0 # for LFT from IMOv1
)

# Print it to see all the current options.  You can change them anytime-
# not just in the constructor.
print(sim_scan)

In [ ]:
# Apply it to simulate the scanning
sim_scan.apply(data)

# Print just the first observation, since there are many
print(data.obs[0])

### Memory Use

We can always see how much memory our data container is using with a small helper operator:

In [ ]:
mem_count = toast.ops.MemoryCounter()
mem_count.apply(data)

## Observation Data

In the last cell you can see that the `Observation` has several "shared" data fields containing the pointing information and some other empty types of data "detdata" and "intervals".  We can just print these like a numpy array:

In [ ]:
print(data.obs[0].shared["times"])

You can see that the "shared" data buffers are a special kind of array that (if MPI is being used) have only a single copy on each compute node.  You can access individual elements with normal slice notation, or you can get a numpy array view by accessing the `.data` attribute.  For example we can plot them:

In [ ]:
# Plot HWP angle vs time for observation 7
times = data.obs[7].shared["times"]
hwp = data.obs[7].shared["hwp_angle"]

fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(times.data[:100], hwp.data[:100])

plt.show()

The `Observation` class also gives us access to the focalplane properties for this observation:

In [ ]:
# The telescope for this observation
print(data.obs[0].telescope)

In [ ]:
# The focalplane
print(data.obs[0].telescope.focalplane)

In [ ]:
# The Table of detector properties
print(data.obs[0].telescope.focalplane.detector_data.info)
print(data.obs[0].telescope.focalplane.detector_data)

## Simulating Detector Signal

As a quick test, we can simulate some timestream components.  First we simulate an orbital plus solarsystem dipole.

In [ ]:
sim_dipole = toast.ops.SimDipole(
    freq=40.0 * u.GHz,
    mode="solar",
)
sim_dipole.apply(data)

Now we can see what was created in the first observation:

In [ ]:
print(data.obs[0])

Notice that the "detdata" attribute now has an item called "signal", which has 12 detector timestreams.  The dipole timestream we simulated is in Kelvin, but let's work in
uK instead.  We can modify the data in place:

In [ ]:
for ob in data.obs:
    ob.detdata["signal"][:, :] *= 1e6

Now we can simulate another component and accumulate that.  Before simulating some instrumental noise, we need to create a "noise model" which describes the noise properties of the detectors.  We could make this by hand with a mixing matrix that included correlations between detectors.  For now, we will use a small operator to just create the noise model from the nominal focalplane properties:

In [ ]:
# Create an uncorrelated noise model from focalplane detector properties

# (print help for this operator)
#?toast.ops.DefaultNoiseModel

In [ ]:
default_model = toast.ops.DefaultNoiseModel(
    noise_model="noise_model" # The string where this will be stored in every observation
)
default_model.apply(data)
print(data.obs[0]["noise_model"])

Next we create an operator that uses this noise model to simulate timestreams.

In [ ]:
# (print help for this operator)
#?toast.ops.SimNoise

In [ ]:
# Create it and specify the noise model to use and the detdata name of the output signal

sim_noise = toast.ops.SimNoise(
    noise_model="noise_model",
)
sim_noise.apply(data)

We can now dig deeper into the simulated fake detector signal so far:

In [ ]:
print(data.obs[0].detdata["signal"])

This `DetectorData` object allows us to access the data by detector name, detector index, or sample range:

In [ ]:
signal = data.obs[0].detdata["signal"]

In [ ]:
print(signal["L00_003_QA_040T"])

In [ ]:
print(signal[["L00_003_QA_040T", "L00_004_QB_040T"], 0:4])

In [ ]:
# The whole thing...
print(signal[:, :])

The `detdata` attribute of an observation contains just the local data on each process, so you can read and write to these arrays.  One can plot them using the shared timestamps as well:

In [ ]:
times = data.obs[0].shared["times"]

fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(times.data, signal["L00_003_QA_040T"])

plt.show()

## Other Operators

You can piece together many other operators to add different types of signal and systematics, etc.  These can be covered in later notebooks.

## Map Making

Here we show a simple example of mapmaking.  First, we need to create several operators that we will pass to the map maker (which will apply the operators internally).  We begin with the pointing information.  The low-level "detector pointing" operator maps telescope boresight quaternions to detector quaternions.  The "pointing matrix" has 2 parts, the sky pixels and the Stokes weights:

In [ ]:
# The default detector pointing operator just takes the focalplane offset from
# the boresight and applies it to boresight pointing

det_pointing = toast.ops.PointingDetectorSimple()
# Operator traits can be set in the constructor, or afterwards.  The default "names"
# for the different objects in the Observation can be set per-experiment globally,
# and we can also get the traits from earlier operators in the workflow and use
# them in later ones:
det_pointing.boresight = sim_scan.boresight

# This operator defines the sky pixels
pixels = toast.ops.PixelsHealpix(
    nside=512,
    detector_pointing=det_pointing
)

# These are the Stokes weights.  Notice we set some traits in the
# constructor- we could also set these aftwards if we wanted.
weights = toast.ops.StokesWeights(
    mode="IQU",
    hwp_angle=sim_scan.hwp_angle,
    detector_pointing=det_pointing
)

Next we need a "binning" operator that will be used during the template amplitude solving and also the final map binning.  The mapmaker accepts different binning operators for these two steps, but here we use the same one.

Here we demonstrate a useful technique:  If you have a previous operator configured with particular traits (in this case the name of the noise model in the observations), you can access that and pass it to other operators.

In [ ]:
# Set up binning operator for solving and final map.  Again,
# this is just creating the operator, not actually doing
# anything yet.
binner = toast.ops.BinMap(
    pixel_pointing=pixels,
    stokes_weights=weights,
    noise_model=default_model.noise_model,
)

Now we set up the templates we will use in our map making.  In this case we will use a single template for offset amplitudes (destriping baselines).  After configuring our templates, we add them to a template matrix operator:

In [ ]:
# Set up template matrix with just an offset template.

tmpl = toast.templates.Offset(
    times=sim_scan.times,
    noise_model=default_model.noise_model,
    step_time=5.0 * u.second,
)
tmatrix = toast.ops.TemplateMatrix(templates=[tmpl])

### Low Memory Use

Finally we are ready to create our Mapmaking operator and run it.  This will be **slow**, since by default we are not saving the pointing.  So every iteration of the solver we are re-generating the pixel number and Stokes weights for every sample, for every detector.  We are also doing this using a single process in this notebook.

In [ ]:
# Map maker set up

# Where the maps should be written
output_dir = "."

mapper = toast.ops.MapMaker(
    name="mapmaker", # This name will prefix all the output products.
    det_data="signal", # The name of the detector data we simulated
    binning=binner,
    template_matrix=tmatrix,
    solve_rcond_threshold=1.0e-6,
    map_rcond_threshold=1.0e-6,
    iter_max=10,
    output_dir=output_dir
)

# Make the map
mapper.apply(data)

Now we can read our outputs and plot them.  Note that our timestream has the dipole in it still, which we would normally use for calibration and remove. 

In [ ]:
mapfile_root = os.path.join(output_dir, mapper.name)

hits = hp.read_map(
    f"{mapfile_root}_hits.fits", 
    dtype=np.int32
)
hp.mollview(hits, min=0, max=500)

Imap, Qmap, Umap = hp.read_map(
    f"{mapfile_root}_map.fits", 
    field=None
)
hp.mollview(Imap, min=-0.0005, max=0.0005)
hp.mollview(Qmap, min=-0.0005, max=0.0005)
hp.mollview(Umap, min=-0.0005, max=0.0005)

### Larger Memory Use and Faster

If we have enough memory, we can compute the pointing once and save it.  Check how much memory we are using right now:

In [ ]:
# Reset the counter first...
mem_count.total_bytes = 0
mem_count.apply(data)

OK, now we can generate the pointing once for the whole dataset:

In [ ]:
pixels.apply(data)
weights.apply(data)

In [ ]:
# Check memory use now
mem_count.total_bytes = 0
mem_count.apply(data)

Now re-run the mapmaking and write the outputs to a different name:

In [ ]:
mapper.name = "example2"
mapper.apply(data)

So here we see that it runs much faster.